In [1]:
import pandas as pd
df =pd.read_csv('./dataset/train.csv')

In [2]:
df_nh = df.loc[df["parentesco1"]!=1]
#결측값은 0으로 제거해주기 
df_nh = df_nh.fillna(0)

In [3]:
df_ny =pd.DataFrame(df_nh["Target"])
df_ny.head()

,Target
3,4
4,4
6,4
7,4
9,4


In [4]:
#불필요한 피쳐들 제거해 x1값 만들기 
# age, SQBescolari, SQBage, SQBhogar_total, SQBedjefe, SQBhogar_nin, SQBovercrowding,
# SQBdependency, SQBmeaned, agesq

#결과값 y에 해당하는 값 삭제 
df_x2 = df_nh.drop(["Target"],1)
#세대주성별 교육년수 (논의필요)
df_x2 = df_x2.drop(["edjefa", "edjefe"],1) 
#중복정보 제거
df_x2 = df_x2.drop(["dependency","female","area2","hacdor","hacapo","bedrooms","r4h3","r4m3"],1) 
#수학적으로 의미가 없는 값 제거
df_x2 = df_x2.drop(["Id","SQBescolari", "SQBage", "SQBhogar_total", "SQBedjefe", "SQBhogar_nin", "SQBovercrowding","idhogar"],1) 
#세대주와의 관계열 제거
df_x2 = df_x2.drop(["parentesco1","parentesco2","parentesco3","parentesco4","parentesco5","parentesco6","parentesco7","parentesco8",
                   "parentesco9","parentesco10","parentesco11","parentesco12"],1)
#집세 임시제거!!!!!!!!!!!!!!!!!!!!!!!!!!
df_x2 = df_x2.drop(["v2a1"],1)
df_x2.head()

,rooms,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4m1,r4m2,r4t1,...,lugar2,lugar3,lugar4,lugar5,lugar6,area1,age,SQBdependency,SQBmeaned,agesq
3,5,1,1,1,1.0,0,2,1,1,1,...,0,0,0,0,0,1,17,1.0,121.0,289
4,5,1,1,1,1.0,0,2,1,1,1,...,0,0,0,0,0,1,37,1.0,121.0,1369
6,5,1,1,1,1.0,0,2,1,1,1,...,0,0,0,0,0,1,8,1.0,121.0,64
7,2,1,1,0,0.0,0,1,2,1,2,...,0,0,0,0,0,1,7,1.0,100.0,49
9,2,1,1,0,0.0,0,1,2,1,2,...,0,0,0,0,0,1,28,1.0,100.0,784


In [5]:
#새로운열 추가 
# df_x1['rent_to_rooms'] = df_x1['v2a1']/df_x1['rooms']
# df_x1['r4t3_to_rooms'] = df_x1['r4t3']/df_x1['rooms']
# df_x1['rent_to_r4t3'] = df_x1['v2a1']/df_x1['r4t3']
# df_x1['v2a1_to_r4t3'] = df_x1['v2a1']/(df_x1['r4t3'] - df_x1['r4t1'])
df_x2['lent'] = df_x2['tamviv']-df_x2['tamhog']

#열 생성 이후 불필요한 열 제거 
df_x2 = df_x2.drop(["r4t1","r4t2","r4t3","tamhog","tamviv"],1) 
df_x2.head()

,rooms,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4m1,r4m2,escolari,...,lugar3,lugar4,lugar5,lugar6,area1,age,SQBdependency,SQBmeaned,agesq,lent
3,5,1,1,1,1.0,0,2,1,1,9,...,0,0,0,0,1,17,1.0,121.0,289,0
4,5,1,1,1,1.0,0,2,1,1,11,...,0,0,0,0,1,37,1.0,121.0,1369,0
6,5,1,1,1,1.0,0,2,1,1,2,...,0,0,0,0,1,8,1.0,121.0,64,0
7,2,1,1,0,0.0,0,1,2,1,0,...,0,0,0,0,1,7,1.0,100.0,49,0
9,2,1,1,0,0.0,0,1,2,1,11,...,0,0,0,0,1,28,1.0,100.0,784,0


In [6]:
from tqdm import tqdm_notebook
import tensorflow as tf
import numpy as np

C:\Python\Anaconda3-52\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        # 6 was used in the paper.
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the same limints as above since this repicks
        # values greater than 2 standard deviations from the mean.
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [8]:
df_ny["Target"]=df_ny["Target"]-1

In [9]:
# Review : Learning rate and Evaluation
import tensorflow as tf
import random
import matplotlib.pyplot as plt

x_n_data = x_n_data = df_x2
sess = tf.Session()
y_n_data = tf.one_hot(df_ny, depth = 4).eval(session=sess)
y_n_data = tf.reshape(y_n_data, shape=[-1,4]).eval(session=sess)
print(y_n_data)
tf.set_random_seed(999)  # reproducibility


# parameters
learning_rate = 0.001



X = tf.placeholder(tf.float32, [None, 107])
Y = tf.placeholder(tf.float32, [None, 4])

W1 = tf.get_variable("W1", shape=[107, 64],
                     initializer=xavier_init(107, 64))
b1 = tf.Variable(tf.random_normal([64]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[64, 64],
                     initializer=xavier_init(64, 64))
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[64, 4],
                     initializer=xavier_init(64, 4))
b3 = tf.Variable(tf.random_normal([4]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


In [10]:
# Launch the graph in a session.
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(10001):
    sess.run(optimizer, feed_dict={X: x_n_data, Y: y_n_data})
    if step % 1000 == 0 or step < 100:
        loss, acc = sess.run([cost, accuracy], feed_dict={
                             X: x_n_data, Y: y_n_data})
        print("Step: {:5}, \t Loss: {:.3f}, \t Acc: {:.2%}".format(
            step, loss, acc))

Step:     0, 	 Loss: 94.067, 	 Acc: 8.66%
Step:     1, 	 Loss: 76.042, 	 Acc: 10.21%
Step:     2, 	 Loss: 59.346, 	 Acc: 12.47%
Step:     3, 	 Loss: 46.308, 	 Acc: 15.98%
Step:     4, 	 Loss: 35.626, 	 Acc: 16.87%
Step:     5, 	 Loss: 23.247, 	 Acc: 17.89%
Step:     6, 	 Loss: 13.969, 	 Acc: 48.31%
Step:     7, 	 Loss: 17.486, 	 Acc: 60.65%
Step:     8, 	 Loss: 20.402, 	 Acc: 61.53%
Step:     9, 	 Loss: 22.283, 	 Acc: 61.48%
Step:    10, 	 Loss: 23.238, 	 Acc: 61.45%
Step:    11, 	 Loss: 23.477, 	 Acc: 61.45%
Step:    12, 	 Loss: 23.134, 	 Acc: 61.45%
Step:    13, 	 Loss: 22.374, 	 Acc: 61.47%
Step:    14, 	 Loss: 21.199, 	 Acc: 61.45%
Step:    15, 	 Loss: 19.651, 	 Acc: 61.44%
Step:    16, 	 Loss: 17.791, 	 Acc: 61.25%
Step:    17, 	 Loss: 15.686, 	 Acc: 59.83%
Step:    18, 	 Loss: 13.400, 	 Acc: 57.26%
Step:    19, 	 Loss: 10.973, 	 Acc: 53.52%
Step:    20, 	 Loss: 8.430, 	 Acc: 50.65%
Step:    21, 	 Loss: 5.796, 	 Acc: 48.80%
Step:    22, 	 Loss: 4.289, 	 Acc: 26.32%
Step:    23, 	 

In [11]:
df2 =pd.read_csv('./dataset/test.csv')

In [12]:
df2_h = df2.fillna(0)

In [13]:
#결과값 y에 해당하는 값 삭제 
df2_x1 = df2_h
#세대주성별 교육년수 (논의필요)
df2_x1 = df2_x1.drop(["edjefa", "edjefe"],1) 
#중복정보 제거
df2_x1 = df2_x1.drop(["dependency","female","area2","hacdor","hacapo","bedrooms","r4h3","r4m3"],1) 
#수학적으로 의미가 없는 값 제거
df2_x1 = df2_x1.drop(["Id","SQBescolari", "SQBage", "SQBhogar_total", "SQBedjefe", "SQBhogar_nin", "SQBovercrowding","idhogar"],1) 
#세대주와의 관계열 제거
df2_x1 = df2_x1.drop(["parentesco1","parentesco2","parentesco3","parentesco4","parentesco5","parentesco6","parentesco7","parentesco8",
                   "parentesco9","parentesco10","parentesco11","parentesco12"],1)
df2_x1['lent'] = df2_x1['tamviv']-df2_x1['tamhog']
#집세 임시제거!!!!!!!!!!!!!!!!!!!!!!!!!!
df2_x1 = df2_x1.drop(["v2a1"],1)
df2_x1 = df2_x1.drop(["r4t1","r4t2","r4t3","tamhog","tamviv"],1) 
##새로운변수 추가 

In [14]:
df2_x1 =  df2_x1.values.tolist()

In [15]:
# Testing & One-hot encoding
test_data = df2_x1


pred_val = sess.run(hypothesis, feed_dict={X: test_data})
pred_idx = sess.run(tf.argmax(pred_val, 1))

# print("predict value : \n {} \n\npredict index : {}".format(pred_val, pred_idx))
print("test data : {} \n\npredict value : \n {} \n\npredict index : {}".format(test_data, pred_val, pred_idx))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
submission = pd.DataFrame({'Id' : df2.Id, 'Target' : pred_idx})
submission.head()

,Id,Target
0,ID_2f6873615,3
1,ID_1c78846d2,3
2,ID_e5442cf6a,3
3,ID_a8db26a79,3
4,ID_a62966799,3


In [17]:
submission.to_csv("submissions.csv", index =False)